# To-Do List AI Agent

This project is a simple, interactive AI agent built using LangGraph that helps users manage their to-do tasks via the command line. It uses LangGraph's stateful workflow design to continuously prompt the user, process input, and update a shared task list.

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, List

In [2]:
# Load environment variables from .env file
load_dotenv()

False

In [3]:
# Initialize the language model (LLM) — placeholder for future enhancements
llm = ChatOpenAI(model="gpt-4", temperature=0)

Users can perform five main actions:

-> Add new tasks

-> List all tasks

-> Mark tasks as completed

-> Delete tasks

-> Exit the agent

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# Define the shape of the shared state across graph nodes
class TaskState(TypedDict):
    tasks: List[dict]            # List to store tasks with 'task' and 'completed' fields
    message: Optional[str]       # Stores the current action (e.g., add, list, etc.)

# This will store all tasks (shared outside LangGraph to allow modification)
task_list = []

The core logic is built using two custom nodes:

ask_action – prompts the user for input

handle_action – processes the action and updates state

In [5]:
# === NODE 1: Ask the user what action they want to perform ===
def ask_action(state: TaskState):
    print("\nWhat would you like to do?")
    print("Options: add, list, complete, delete, exit")
    action = input("Action: ").lower()
    state["message"] = action  # Update state with user input
    return state

In [6]:
# === NODE 2: Handle the user's action ===
def handle_action(state: TaskState):
    action = state["message"]

    if action == "add":
        task = input("Enter the task: ")
        task_list.append({"task": task, "completed": False})
        print(f"✅ Task added: {task}")

    elif action == "list":
        if not task_list:
            print("🗒 No tasks yet.")
        else:
            for i, t in enumerate(task_list):
                status = "✅" if t["completed"] else "⏳"
                print(f"{i + 1}. {t['task']} - {status}")

    elif action == "complete":
        index = int(input("Enter task number to complete: ")) - 1
        if 0 <= index < len(task_list):
            task_list[index]["completed"] = True
            print("🎯 Task marked as complete.")
        else:
            print("❌ Invalid task number.")

    elif action == "delete":
        index = int(input("Enter task number to delete: ")) - 1
        if 0 <= index < len(task_list):
            removed = task_list.pop(index)
            print(f"🗑 Removed: {removed['task']}")
        else:
            print("❌ Invalid task number.")

    elif action == "exit":
        print("👋 Exiting Task Manager Agent.")
        # No need to call `exit()` anymore — we will handle the transition
        state["message"] = "exit"
        return state

    else:
        print("❓ Unknown command. Try again.")

    return state

## Graph Construction with LangGraph

The graph loops back to keep the agent running until the user exits, showcasing the LangGraph architecture of nodes, edges, and persistent state.

In [7]:
# Initialize the LangGraph with the TaskState schema
graph = StateGraph(TaskState)

# Register custom nodes (functions)
graph.add_node("ask_action", ask_action)
graph.add_node("handle_action", handle_action)

# Define the starting point of the agent
graph.set_entry_point("ask_action")

# Define transitions (edges) between nodes
graph.add_edge("ask_action", "handle_action")

# 🔁 Replace this line:
# graph.add_edge("handle_action", "ask_action")

# ✅ With conditional exit logic:
def next_node_selector(state: TaskState):
    if state["message"] == "exit":
        return END
    return "ask_action"

graph.add_conditional_edges("handle_action", next_node_selector)

# Compile the graph into an executable agent
agent = graph.compile()

# Initial state passed to the agent
agent.invoke({"tasks": []})


What would you like to do?
Options: add, list, complete, delete, exit
Action: add
Enter the task: Finish the LangGraph project
✅ Task added: Finish the LangGraph project

What would you like to do?
Options: add, list, complete, delete, exit
Action: add
Enter the task: Submit for grading
✅ Task added: Submit for grading

What would you like to do?
Options: add, list, complete, delete, exit
Action: list
1. Finish the LangGraph project - ⏳
2. Submit for grading - ⏳

What would you like to do?
Options: add, list, complete, delete, exit
Action: complete
Enter task number to complete: 1
🎯 Task marked as complete.

What would you like to do?
Options: add, list, complete, delete, exit
Action: list
1. Finish the LangGraph project - ✅
2. Submit for grading - ⏳

What would you like to do?
Options: add, list, complete, delete, exit
Action: complete
Enter task number to complete: 2
🎯 Task marked as complete.

What would you like to do?
Options: add, list, complete, delete, exit
Action: list
1. Fin

{'tasks': [], 'message': 'exit'}